## Exercice 1 - Entraînement, exportation d'un CNN et versioning

L'objectif de ce TP est de développer, entraîner et exporter un modèle de type CNN en utilisant le dataset cats VS dogs pour la classification d'images.

1. Donnez les informations quant au.x GPU.s utilisé.s (nombre, type, mémoire dédiée)
2. Renseignez la version de Python utilisée
3. Chargez le dataset cats_vs_dogs avec TensorFlow
4. Séparez le dataset de la manière suivante :
5. Implémentez un modèle de classification d'images avec Tensorflow
6. Entraînez le modèle en utilisant la puissance du GPU
7. Testez le modèle sur le dataset de test
8. Sauvegardez le modèle au format ONNX
9. Chargez-le pour effectuer une prédiction sur une image
10. Créez un fichier requirements.txt avec toutes les dépendances installées et leur version

#### Install dependencies

In [ ]:
pip install numpy tensorflow_datasets

In [ ]:
pip install tensorflow[and-cuda] --upgrade

#### Import Python dependencies

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### 1. Informations sur le GPU et la version de Python

In [4]:
!nvidia-smi

Wed Feb 12 21:08:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100S-PCIE-32GB          On  |   00000000:00:06.0 Off |                    0 |
| N/A   33C    P0             25W /  250W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 2. Chargement du dataset Cats vs Dogs depuis TensorFlow Datasets

Accéder au dataset : https://www.tensorflow.org/datasets/catalog/cats_vs_dogs

In [6]:
dataset, info = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)

In [7]:
print("Dataset :", dataset)

Dataset : {'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [8]:
print("Info :", info)

Info : tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/workspace/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, John (JD) and Howell, 

### 3. Séparation en 70% entraînement, 20% validation, 10% test

- 70 % - training set
- 20% - validation set
- 10% - test set

In [9]:
total_size = info.splits['train'].num_examples
print("Total Size :", total_size)

train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

Total Size : 23262


In [10]:
train_data = dataset['train'].take(train_size)
val_data = dataset['train'].skip(train_size).take(val_size)
test_data = dataset['train'].skip(train_size + val_size).take(test_size)

In [11]:
print("Training Size :", train_size, \
      "\nValidation Size :", val_size, \
      "\nTest Size :", test_size)

Training Size : 16283 
Validation Size : 4652 
Test Size : 2327


### 4. Prétraitement des images

In [12]:
# Prétraitement des images
def preprocess(image, label):
    image = tf.image.resize(image, (128, 128)) / 255.0
    return image, label

In [13]:
train_data = train_data.map(preprocess).batch(32).shuffle(1000)
val_data = val_data.map(preprocess).batch(32)
test_data = test_data.map(preprocess).batch(32)

### 5. Implémentez un modèle de classification d'images avec Tensorflow

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

### 6. Entraînez le modèle en utilisant la puissance du GPU

In [16]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, epochs=3, validation_data=val_data)

Epoch 1/3
475/509 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6889 - loss: 0.5860

2025-02-12 21:10:01.173365: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[27,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[27,3,128,128]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-02-12 21:10:01.206737: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[27,64,61,61]{3,2,1,0}, u8[0]{0}) custom-call(f32[27,32,63,63]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

507/509 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6905 - loss: 0.5845

2025-02-12 21:10:06.229645: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,126,126]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,128,128]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-02-12 21:10:06.265430: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,61,61]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,63,63]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

509/509 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.6907 - loss: 0.5844 - val_accuracy: 0.7377 - val_loss: 0.5270
Epoch 2/3
509/509 ━━━━━━━━━━━━━━━━━━━━ 13s 18ms/step - accuracy: 0.7810 - loss: 0.4686 - val_accuracy: 0.7752 - val_loss: 0.4800
Epoch 3/3
509/509 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.8352 - loss: 0.3658 - val_accuracy: 0.7758 - val_loss: 0.4757


### 7. Testez le modèle sur le dataset de test

In [21]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

73/73 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7806 - loss: 0.4630
Test Accuracy: 77.52%


### 8. Sauvegarde du modèle au format ONNX

In [ ]:
pip install onnx onnxruntime-gpu tf2onnx

In [22]:
import onnx
import tf2onnx
import numpy as np
import onnxruntime

In [23]:
model.output_names=['output']

onnx_model_path = "model.onnx"
spec = (tf.TensorSpec((None, 128, 128, 3), tf.float32),)
onxx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=18)
onnx.save_model(onxx_model, onnx_model_path)
print(f"Modèle sauvegardé sous {onnx_model_path}")

I0000 00:00:1739394664.361461   47421 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1739394664.361671   47421 single_machine.cc:361] Starting new session
I0000 00:00:1739394664.363023   47421 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:00:06.0, compute capability: 7.0
I0000 00:00:1739394665.241056   47421 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:00:06.0, compute capability: 7.0
I0000 00:00:1739394665.375544   47421 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
I0000 00:00:1739394665.375764   47421 single_machine.cc:361] Starting new session
I0000 00:00:1739394665.377069   47421 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 

Modèle sauvegardé sous model.onnx


### 9. Chargement du modèle ONNX et prédiction sur une image

In [ ]:
!pip install Pillow

In [26]:
from PIL import Image
onnx_model_path = "model.onnx"
session = onnxruntime.InferenceSession(onnx_model_path)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

image_path = "/workspace/dog.jpeg"
image = Image.open(image_path).resize((128, 128))
image = np.array(image, dtype=np.float32) / 255.0
image = np.expand_dims(image, axis=0)

prediction = session.run([output_name], {input_name: image})
print("Prédiction ONNX:", prediction)

Prédiction ONNX: [array([[0.8292207]], dtype=float32)]


### 10 - Créez un fichier requirements.txt avec toutes les dépendances installées et leur version

In [ ]:
pip list